In [1]:
import time
import argparse
import numpy as np

import torch
import torch.optim as optim

import torch.nn as nn

from utils import load_data,normalize,toy_data,nmi_score,modularity_matrix,modularity
from models import GNN

import community as community_louvain
from networkx import from_numpy_matrix
import networkx as nx

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

from sklearn.cluster import SpectralClustering

import pycombo

import pandas as pd

torch.set_printoptions(sci_mode=False)

from utils import doublerelu

In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
cuda = torch.cuda.is_available()
weight_decay = 10e-4
epochs = 1000
seed = 17

In [4]:
class GNN1Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN1Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size

        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [5]:
class GNN2Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN2Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size
        weight1_eye = torch.FloatTensor(torch.eye(in_features, in_features))
        weight1_eye = weight1_eye.reshape((1, in_features, in_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        weight1_rand = torch.empty(batch_size,in_features,out_features-in_features)
        torch.nn.init.xavier_uniform_(weight1_rand, gain=1.0)
        self.weight1 = Parameter(torch.cat((weight1_eye,weight1_rand),dim=2))
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [6]:
class GNN1(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN1, self).__init__()

        self.gc1 = GNN1Layer(batch_size, ndim, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [7]:
class GNN2(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN2, self).__init__()

        self.gc1 = GNN2Layer(batch_size, ndim, hidden)
        self.gc2 = GNN1Layer(batch_size, hidden, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = doublerelu(self.gc2(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [8]:
def community_detection(G,method,GNN,features=None,hidden=1):
    
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed(seed)
    
    adj = nx.to_numpy_matrix(G)
    adj = np.expand_dims(adj, axis=0)
        
    #Combo
    combo_partition = pycombo.execute(G)
    nb_comm = max(combo_partition[0].values())+1
    combo_mod = combo_partition[1]

    adj_norm = normalize(adj)

    adj = torch.FloatTensor(np.array(adj))
    adj_norm = torch.FloatTensor(np.array(adj_norm))
    
        
    if type(features) == type(None):

        # features
        if method == "louvain":
            partition = community_louvain.best_partition(G)
            #get binary matrix of the partition
            nb_community = max(list(partition.values())) + 1
            communities =  np.array(list(partition.values())).reshape(-1)
            C = np.eye(nb_community)[communities]
            C = torch.FloatTensor(C)
            #perturbations
            C = perturb(C)
        elif method == "spectral":
            partition = spectralPartition(adj,nb_comm)
            C = partition.float()
            nb_community = C.shape[1]

        features = torch.FloatTensor(C)
        features = features.unsqueeze(0)
    
    if method == "louvain":
        lr = 0.001
    elif method == "spectral":
        lr = 0.001
        
    nb_community = features.shape[-1]
    
    Q = modularity_matrix(adj)
    
    # Model and optimizer

    model = GNN(batch_size=adj.shape[0],
                nfeat=adj.shape[1],
                ndim=nb_community,
                hidden=hidden)

    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        Q = Q.cuda()

    # Train model
    t_total = time.time()

    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    

    for epoch in range(epochs):

        t = time.time()
        model.train()
        optimizer.zero_grad()

        C = model(features, adj_norm)
        loss = modularity(C,Q)

        loss.backward(retain_graph=True)

        optimizer.step()

        if epoch == 0:
            best_loss = loss
            init_mod = loss
            best_C = C
        else:
            if loss < best_loss:
                best_loss = loss
                best_C = C

        if epoch % 100 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Modularity: {:.8f}'.format(-best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))

    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return nb_community,-init_mod,-best_loss,combo_mod, best_C

In [9]:
def perturb(A):
    p = 0.4
    epsilon = torch.rand(A.shape).uniform_(0, p) - torch.rand(A.shape).uniform_(0, p)
    return torch.clip(A + epsilon,0,1)

In [10]:
def spectralPartition(A,nb_comm):
    A = A[0] # TODO 3D
    clustering = SpectralClustering(n_clusters=nb_comm,assign_labels='discretize',random_state=0).fit(A)
    clusters = list(clustering.labels_)
    partitions = torch.tensor(clusters)
    return torch.nn.functional.one_hot(partitions)

In [11]:
def exp(hidden):
    G = [nx.karate_club_graph(),nx.les_miserables_graph(),nx.florentine_families_graph()]
    methods = ["louvain","spectral"]
    nb_communities = []
    init_mods = []
    finetuned_mods = []
    combo_mods = []
    methodss = []
    networks = ["Karate Club","Karate Club","Les Miserables","Les Miserables","Florentine","Florentine"]
    hidden_dim = []
    for network in G:
        for method in methods:
            nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN1,hidden=hidden)
            initial_mod = torch.clone(init_mod)
            nb_community,init_mod,finetuned_mod,combo_mod,features = community_detection(network,method,GNN2,features=features,hidden=nb_community+hidden)
            nb_communities.append(nb_community)
            init_mods.append(initial_mod.cpu().detach().numpy()[0][0])
            finetuned_mods.append(finetuned_mod.cpu().detach().numpy()[0][0])
            combo_mods.append(combo_mod)
            methodss.append(method)
            hidden_dim.append(nb_community+hidden)
    df = pd.DataFrame([networks,methodss,nb_communities,hidden_dim,init_mods,finetuned_mods,combo_mods]).transpose()
    df.columns = ["Network","Method","Communities","Hidden Dimensionality","Initial Modularity","Finetuned Moduclarity","Combo Modularity"] 
    return df

In [12]:
dfs = []
for hidden in range(1,11):
    dfs.append(exp(hidden))

Epoch: 0001 Modularity: 0.25446442 time: 0.3600s
Epoch: 0101 Modularity: 0.38794592 time: 0.0020s
Epoch: 0201 Modularity: 0.41832262 time: 0.0020s
Epoch: 0301 Modularity: 0.41978961 time: 0.0020s
Epoch: 0401 Modularity: 0.41978961 time: 0.0020s
Epoch: 0501 Modularity: 0.41978961 time: 0.0020s
Epoch: 0601 Modularity: 0.41978961 time: 0.0020s
Epoch: 0701 Modularity: 0.41978961 time: 0.0020s
Epoch: 0801 Modularity: 0.41978961 time: 0.0010s
Epoch: 0901 Modularity: 0.41978961 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.1514s
Epoch: 0001 Modularity: 0.41978961 time: 0.0030s
Epoch: 0101 Modularity: 0.41978961 time: 0.0030s
Epoch: 0201 Modularity: 0.41978961 time: 0.0040s
Epoch: 0301 Modularity: 0.41978961 time: 0.0040s
Epoch: 0401 Modularity: 0.41978961 time: 0.0040s
Epoch: 0501 Modularity: 0.41978961 time: 0.0030s
Epoch: 0601 Modularity: 0.41978961 time: 0.0030s
Epoch: 0701 Modularity: 0.41978961 time: 0.0030s
Epoch: 0801 Modularity: 0.41978961 time: 0.0030s
Epoch: 0901 Modula

Epoch: 0201 Modularity: 0.39907956 time: 0.0030s
Epoch: 0301 Modularity: 0.39907956 time: 0.0030s
Epoch: 0401 Modularity: 0.39907956 time: 0.0030s
Epoch: 0501 Modularity: 0.39907956 time: 0.0020s
Epoch: 0601 Modularity: 0.39907956 time: 0.0030s
Epoch: 0701 Modularity: 0.39907956 time: 0.0030s
Epoch: 0801 Modularity: 0.39907956 time: 0.0030s
Epoch: 0901 Modularity: 0.39907956 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.5901s
Epoch: 0001 Modularity: 0.26796943 time: 0.0010s
Epoch: 0101 Modularity: 0.53119338 time: 0.0020s
Epoch: 0201 Modularity: 0.56412679 time: 0.0010s
Epoch: 0301 Modularity: 0.56572592 time: 0.0010s
Epoch: 0401 Modularity: 0.56611645 time: 0.0010s
Epoch: 0501 Modularity: 0.56625259 time: 0.0020s
Epoch: 0601 Modularity: 0.56629515 time: 0.0010s
Epoch: 0701 Modularity: 0.56629837 time: 0.0010s
Epoch: 0801 Modularity: 0.56629837 time: 0.0010s
Epoch: 0901 Modularity: 0.56629837 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.2487s
Epoch: 0001 Modu

Epoch: 0401 Modularity: 0.39308828 time: 0.0010s
Epoch: 0501 Modularity: 0.39308828 time: 0.0010s
Epoch: 0601 Modularity: 0.39308828 time: 0.0010s
Epoch: 0701 Modularity: 0.39308828 time: 0.0010s
Epoch: 0801 Modularity: 0.39308828 time: 0.0010s
Epoch: 0901 Modularity: 0.39308828 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.2405s
Epoch: 0001 Modularity: 0.39308828 time: 0.0030s
Epoch: 0101 Modularity: 0.42359781 time: 0.0030s
Epoch: 0201 Modularity: 0.44135314 time: 0.0020s
Epoch: 0301 Modularity: 0.47043595 time: 0.0020s
Epoch: 0401 Modularity: 0.47043595 time: 0.0030s
Epoch: 0501 Modularity: 0.47043595 time: 0.0020s
Epoch: 0601 Modularity: 0.47043595 time: 0.0020s
Epoch: 0701 Modularity: 0.47043595 time: 0.0020s
Epoch: 0801 Modularity: 0.47043595 time: 0.0040s
Epoch: 0901 Modularity: 0.47043595 time: 0.0030s
Optimization Finished!
Total time elapsed: 2.2946s
Epoch: 0001 Modularity: 0.22756433 time: 0.0010s
Epoch: 0101 Modularity: 0.36726603 time: 0.0010s
Epoch: 0201 Modu

Epoch: 0601 Modularity: 0.39875001 time: 0.0020s
Epoch: 0701 Modularity: 0.39875001 time: 0.0020s
Epoch: 0801 Modularity: 0.39875001 time: 0.0020s
Epoch: 0901 Modularity: 0.39875001 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.3227s
Epoch: 0001 Modularity: -0.17124999 time: 0.0010s
Epoch: 0101 Modularity: -0.04291525 time: 0.0020s
Epoch: 0201 Modularity: 0.03025404 time: 0.0010s
Epoch: 0301 Modularity: 0.13500090 time: 0.0010s
Epoch: 0401 Modularity: 0.21293184 time: 0.0010s
Epoch: 0501 Modularity: 0.21866006 time: 0.0010s
Epoch: 0601 Modularity: 0.22078627 time: 0.0010s
Epoch: 0701 Modularity: 0.22193506 time: 0.0010s
Epoch: 0801 Modularity: 0.22437158 time: 0.0010s
Epoch: 0901 Modularity: 0.24251574 time: 0.0010s
Optimization Finished!
Total time elapsed: 1.1968s
Epoch: 0001 Modularity: 0.29674834 time: 0.0030s
Epoch: 0101 Modularity: 0.29745692 time: 0.0020s
Epoch: 0201 Modularity: 0.29866636 time: 0.0020s
Epoch: 0301 Modularity: 0.29957303 time: 0.0020s
Epoch: 0401 Mo

Epoch: 0801 Modularity: 0.41978961 time: 0.0020s
Epoch: 0901 Modularity: 0.41978961 time: 0.0020s
Optimization Finished!
Total time elapsed: 1.6257s
Epoch: 0001 Modularity: 0.41978961 time: 0.0020s
Epoch: 0101 Modularity: 0.41978961 time: 0.0020s
Epoch: 0201 Modularity: 0.41978961 time: 0.0040s
Epoch: 0301 Modularity: 0.41978961 time: 0.0030s
Epoch: 0401 Modularity: 0.41978961 time: 0.0030s
Epoch: 0501 Modularity: 0.41978961 time: 0.0030s
Epoch: 0601 Modularity: 0.41978961 time: 0.0030s
Epoch: 0701 Modularity: 0.41978961 time: 0.0030s
Epoch: 0801 Modularity: 0.41978961 time: 0.0030s
Epoch: 0901 Modularity: 0.41978961 time: 0.0030s
Optimization Finished!
Total time elapsed: 3.0508s
Epoch: 0001 Modularity: 0.34015450 time: 0.0020s
Epoch: 0101 Modularity: 0.35263938 time: 0.0010s
Epoch: 0201 Modularity: 0.35894844 time: 0.0020s
Epoch: 0301 Modularity: 0.36297059 time: 0.0010s
Epoch: 0401 Modularity: 0.36976776 time: 0.0020s
Epoch: 0501 Modularity: 0.39869636 time: 0.0010s
Epoch: 0601 Modu

Optimization Finished!
Total time elapsed: 2.8524s
Epoch: 0001 Modularity: 0.26796943 time: 0.0030s
Epoch: 0101 Modularity: 0.53119338 time: 0.0010s
Epoch: 0201 Modularity: 0.56412679 time: 0.0020s
Epoch: 0301 Modularity: 0.56572592 time: 0.0020s
Epoch: 0401 Modularity: 0.56611645 time: 0.0020s
Epoch: 0501 Modularity: 0.56625259 time: 0.0020s
Epoch: 0601 Modularity: 0.56629515 time: 0.0020s
Epoch: 0701 Modularity: 0.56629837 time: 0.0020s
Epoch: 0801 Modularity: 0.56629837 time: 0.0020s
Epoch: 0901 Modularity: 0.56629837 time: 0.0020s
Optimization Finished!
Total time elapsed: 1.9129s
Epoch: 0001 Modularity: 0.56629837 time: 0.0030s
Epoch: 0101 Modularity: 0.56629837 time: 0.0040s
Epoch: 0201 Modularity: 0.56629837 time: 0.0040s
Epoch: 0301 Modularity: 0.56629837 time: 0.0030s
Epoch: 0401 Modularity: 0.56629837 time: 0.0040s
Epoch: 0501 Modularity: 0.56629837 time: 0.0040s
Epoch: 0601 Modularity: 0.56629837 time: 0.0040s
Epoch: 0701 Modularity: 0.56629837 time: 0.0030s
Epoch: 0801 Modu

Epoch: 0101 Modularity: 0.42705655 time: 0.0020s
Epoch: 0201 Modularity: 0.47577626 time: 0.0020s
Epoch: 0301 Modularity: 0.47577626 time: 0.0030s
Epoch: 0401 Modularity: 0.47577626 time: 0.0020s
Epoch: 0501 Modularity: 0.47577626 time: 0.0030s
Epoch: 0601 Modularity: 0.47577626 time: 0.0040s
Epoch: 0701 Modularity: 0.47577626 time: 0.0030s
Epoch: 0801 Modularity: 0.47577626 time: 0.0020s
Epoch: 0901 Modularity: 0.47577626 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.3148s
Epoch: 0001 Modularity: 0.22756433 time: 0.0020s
Epoch: 0101 Modularity: 0.36726603 time: 0.0010s
Epoch: 0201 Modularity: 0.39875001 time: 0.0010s
Epoch: 0301 Modularity: 0.39875001 time: 0.0010s
Epoch: 0401 Modularity: 0.39875001 time: 0.0020s
Epoch: 0501 Modularity: 0.39875001 time: 0.0010s
Epoch: 0601 Modularity: 0.39875001 time: 0.0010s
Epoch: 0701 Modularity: 0.39875001 time: 0.0020s
Epoch: 0801 Modularity: 0.39875001 time: 0.0010s
Epoch: 0901 Modularity: 0.39875001 time: 0.0010s
Optimization Finis

Epoch: 0401 Modularity: 0.21293184 time: 0.0010s
Epoch: 0501 Modularity: 0.21866006 time: 0.0010s
Epoch: 0601 Modularity: 0.22078627 time: 0.0010s
Epoch: 0701 Modularity: 0.22193506 time: 0.0010s
Epoch: 0801 Modularity: 0.22437158 time: 0.0010s
Epoch: 0901 Modularity: 0.24251574 time: 0.0020s
Optimization Finished!
Total time elapsed: 1.2068s
Epoch: 0001 Modularity: 0.29674834 time: 0.0020s
Epoch: 0101 Modularity: 0.29741162 time: 0.0030s
Epoch: 0201 Modularity: 0.29834443 time: 0.0020s
Epoch: 0301 Modularity: 0.29951960 time: 0.0020s
Epoch: 0401 Modularity: 0.29951960 time: 0.0020s
Epoch: 0501 Modularity: 0.29951960 time: 0.0020s
Epoch: 0601 Modularity: 0.29951960 time: 0.0020s
Epoch: 0701 Modularity: 0.29951960 time: 0.0020s
Epoch: 0801 Modularity: 0.29951960 time: 0.0020s
Epoch: 0901 Modularity: 0.29951960 time: 0.0020s
Optimization Finished!
Total time elapsed: 2.2699s
Epoch: 0001 Modularity: 0.25446442 time: 0.0010s
Epoch: 0101 Modularity: 0.38794592 time: 0.0020s
Epoch: 0201 Modu

In [13]:
dfs[0]

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,5,0.254464,0.419790,0.419790
1,Karate Club,spectral,4,5,0.340154,0.399080,0.419790
2,Les Miserables,louvain,6,7,0.267969,0.566298,0.566688
3,Les Miserables,spectral,6,7,0.117456,0.470180,0.566688
4,Florentine,louvain,3,4,0.227564,0.398750,0.398750
5,Florentine,spectral,3,4,-0.171250,0.299668,0.398750


In [14]:
dfs[1]

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,6,0.254464,0.419790,0.419790
1,Karate Club,spectral,4,6,0.340154,0.399080,0.419790
2,Les Miserables,louvain,6,8,0.267969,0.566298,0.566688
3,Les Miserables,spectral,6,8,0.117456,0.469262,0.562673
4,Florentine,louvain,3,5,0.227564,0.398750,0.398750
5,Florentine,spectral,3,5,-0.171250,0.299621,0.398750


In [15]:
dfs[2]

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,7,0.254464,0.419790,0.419790
1,Karate Club,spectral,4,7,0.340154,0.399080,0.419790
2,Les Miserables,louvain,6,9,0.267969,0.566298,0.566688
3,Les Miserables,spectral,6,9,0.117456,0.470436,0.566688
4,Florentine,louvain,3,6,0.227564,0.398750,0.398750
5,Florentine,spectral,3,6,-0.171250,0.299630,0.398750


In [16]:
dfs[3]

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,8,0.254464,0.419790,0.419790
1,Karate Club,spectral,4,8,0.340154,0.399080,0.419790
2,Les Miserables,louvain,6,10,0.267969,0.566298,0.562673
3,Les Miserables,spectral,6,10,0.117456,0.474391,0.566688
4,Florentine,louvain,3,7,0.227564,0.398750,0.398750
5,Florentine,spectral,3,7,-0.171250,0.299573,0.398750


In [17]:
dfs[4]

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,9,0.254464,0.419790,0.419790
1,Karate Club,spectral,4,9,0.340154,0.399080,0.419790
2,Les Miserables,louvain,6,11,0.267969,0.566298,0.566688
3,Les Miserables,spectral,6,11,0.117456,0.469682,0.566688
4,Florentine,louvain,3,8,0.227564,0.398750,0.398750
5,Florentine,spectral,3,8,-0.171250,0.299517,0.398750


In [18]:
dfs[5]

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,10,0.254464,0.419790,0.419790
1,Karate Club,spectral,4,10,0.340154,0.399080,0.419790
2,Les Miserables,louvain,6,12,0.267969,0.566298,0.566688
3,Les Miserables,spectral,6,12,0.117456,0.476452,0.566688
4,Florentine,louvain,3,9,0.227564,0.398750,0.398750
5,Florentine,spectral,3,9,-0.171250,0.299610,0.398750


In [19]:
dfs[6]

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,11,0.254464,0.419790,0.419790
1,Karate Club,spectral,4,11,0.340154,0.399080,0.419790
2,Les Miserables,louvain,6,13,0.267969,0.566298,0.566688
3,Les Miserables,spectral,5,12,0.164235,0.435793,0.566688
4,Florentine,louvain,3,10,0.227564,0.398750,0.398750
5,Florentine,spectral,3,10,-0.171250,0.299503,0.398750


In [20]:
dfs[7]

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,12,0.254464,0.419790,0.419790
1,Karate Club,spectral,4,12,0.340154,0.399080,0.419790
2,Les Miserables,louvain,6,14,0.267969,0.566298,0.566688
3,Les Miserables,spectral,6,14,0.117456,0.475776,0.562673
4,Florentine,louvain,3,11,0.227564,0.398750,0.398750
5,Florentine,spectral,3,11,-0.171250,0.299386,0.398750


In [21]:
dfs[8]

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,13,0.254464,0.419790,0.419790
1,Karate Club,spectral,4,13,0.340154,0.399080,0.419790
2,Les Miserables,louvain,6,15,0.267969,0.566298,0.562673
3,Les Miserables,spectral,6,15,0.117456,0.477209,0.566688
4,Florentine,louvain,3,12,0.227564,0.398750,0.398750
5,Florentine,spectral,3,12,-0.171250,0.299520,0.398750


In [22]:
dfs[9]

,Network,Method,Communities,Hidden Dimensionality,Initial Modularity,Finetuned Moduclarity,Combo Modularity
0,Karate Club,louvain,4,14,0.254464,0.419790,0.419790
1,Karate Club,spectral,4,14,0.340154,0.399080,0.419790
2,Les Miserables,louvain,6,16,0.267969,0.566298,0.566688
3,Les Miserables,spectral,6,16,0.117456,0.475486,0.566688
4,Florentine,louvain,3,13,0.227564,0.398750,0.398750
5,Florentine,spectral,3,13,-0.171250,0.299535,0.398750
